<a href="https://colab.research.google.com/github/HyeonGeunY/papers/blob/main/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os
from typing import List

In [9]:
class ShortcutBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, stride: int):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, stride=stride, kernel_size=1)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x: torch.Tensor):
        return self.bn(self.conv(x))

In [10]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, stride: int = 1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.act1 = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Identity()
        if (stride != 1) or (in_channels != out_channels):
            self.shortcut = ShortcutBlock(in_channels=in_channels, out_channels=out_channels, stride=stride)
        
        self.act2 = nn.ReLU()

    def forward(self, x: torch.Tensor):
        shortcut = self.shortcut(x)
        x = self.bn2(self.conv2(self.act1(self.bn1(self.conv1(x)))))
        return self.act2(x + shortcut)


In [11]:
class BottleNeckBlock(nn.Module):
    def __init__(self, in_channels, bottle_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, bottle_channels, kernel_size=1, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(bottle_channels)
        self.act1 = nn.ReLU()

        self.conv2 = nn.Conv2d(bottle_channels, bottle_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(bottle_channels)
        self.act2 = nn.ReLU()

        self.conv3 = nn.Conv2d(bottle_channels, out_channels, kernel_size=1, stride=1)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.act3 = nn.ReLU()

        self.shortcut = nn.Identity()
        if stride != 1 or (in_channels != out_channels):
            self.shortcut = ShortcutBlock(in_channels, out_channels, stride=stride)

    def forward(self, x:torch.Tensor):
        shortcut = self.shortcut(x)
        x = self.act1(self.bn1(self.conv1(x)))
        x = self.act2(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        return self.act3(x + shortcut)

In [16]:
class ResNet(nn.Module):
    def __init__(self, num_blocks: List[int], strides : List[int], channels: List[int], first_kernel_size: int = 7, image_channels: int = 3, n_classes=10, bottleneck: List[int] = None):
        """
        MNIST 크기 (28 * 28)에 맞춰주기 위해 stride 크기 조정
        """
        super().__init__()
        self.firstconv = nn.Conv2d(image_channels, channels[0], kernel_size=first_kernel_size, stride=strides[0], padding=first_kernel_size//2)
        self.bn1 = nn.BatchNorm2d(channels[0])
        self.act1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=3)
        self.bottleneck = bottleneck
        
        self.layers = []
        for i in range(1, len(channels)):
            if self.bottleneck:
                self.layers += self._repeat_bottleneck_layers(channels[i - 1], bottleneck[i - 1], channels[i], stride=strides[i], n_count=num_blocks[i - 1])
            else:
                self.layers += self._repeat_layers(channels[i - 1], channels[i], stride=strides[i], n_count=num_blocks[i - 1])

        self.layers = nn.Sequential(*self.layers)
        self.fc = nn.Linear(channels[i], n_classes)
    

    def _repeat_bottleneck_layers(self, in_channels, bottle_channels, out_channels, stride, n_count):
        layers = []
        strides = [stride] + [1] * (n_count - 1)

        for i, s in enumerate(strides):
            if i == 0:
                layers.append(BottleNeckBlock(in_channels, bottle_channels, out_channels, s))
            else:
                layers.append(BottleNeckBlock(out_channels, bottle_channels, out_channels, s))
                
        return layers
    

    def _repeat_layers(self, in_channels, out_channels, stride, n_count):
        layers = []
        strides = [stride] + [1] * (n_count - 1)
        for i, s in enumerate(strides):
            if i == 0:
                layers.append(ResidualBlock(in_channels, out_channels, s))
            else:
                layers.append(ResidualBlock(out_channels, out_channels, s))

        return layers


    def forward(self, x):
        x = self.act1(self.bn1(self.firstconv(x)))
        x = self.maxpool1(x)
        x = self.layers(x)
        x = x.view(x.size(0), x.size(1), -1) # 2차원으로 만들어주기
        x = x.mean(dim=-1)
        x = self.fc(x)
        return x

def resnet18_for_mnist():
    return ResNet(num_blocks=[2, 2, 2, 2], strides=[1, 1, 2, 2, 2], channels=[64, 64, 128, 256, 512], image_channels=1)

# 학습

In [17]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# 훈련 & 추론 함수

In [18]:
device = 'cuda'

net = resnet18_for_mnist()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.01
file_name = 'resnet18_mnist.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0.0002) # weight_decay: L2 norm


def train(epoch):
    """
    train code referenced from
    https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_MNIST_Train.ipynb
    """
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device) # 모델과 데이터를 같은 머신에 탑재
        optimizer.zero_grad()

        benign_outputs = net(inputs) # benign_outputs (B, C)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1) # 값, 인덱스 반환

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item() # eq: 같은지 비교
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval() # 평가 모드로 전환
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device) # 같은 머신에 올리기
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 5:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [19]:
for epoch in range(0, 10):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Current batch: 0
Current benign train accuracy: 0.046875
Current benign train loss: 2.4314069747924805

Current batch: 100
Current benign train accuracy: 0.9609375
Current benign train loss: 0.1369875967502594

Current batch: 200
Current benign train accuracy: 0.96875
Current benign train loss: 0.11663743853569031

Current batch: 300
Current benign train accuracy: 0.96875
Current benign train loss: 0.1703571081161499

Current batch: 400
Current benign train accuracy: 0.984375
Current benign train loss: 0.06594353169202805

Total benign train accuarcy: 90.59
Total benign train loss: 148.6219130679965

[ Test epoch: 0 ]

Test accuarcy: 93.02
Test average loss: 0.002262835589516908
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.9375
Current benign train loss: 0.18407894670963287

Current batch: 100
Current benign train accuracy: 0.9140625
Current benign train loss: 0.24108824133872986

Current batch: 200
Current benign train accuracy: 0.984375
Current